In [16]:
import pandas as pd
with open('weather.txt') as f:
    data = f.readlines()

data2010 = []
    
for line in data:
    if "2010" in line[0:21]:
        data2010.append(line)
    
print(data2010[0])


MX000017004201001TMAX-9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999     278  S-9999   



In [25]:
alldata = []
for month in data2010:
    if "PRCP" not in month:
        essentials = ["MX17004",2010,month[15:17],month[17:21]]
        mondata = month[21:].split()
        alldata.append(essentials + mondata)
print(alldata[0])
df = pd.DataFrame(alldata,columns=["id","year","month","element","d1","d2","d3","d4","d5","d6","d7",
                                   "d8","d9","d10","d11","d12","d13","d14","d15","d16","d17","d18","d19","d20",
                                   "d21","d22","d23","d24","d25","d26","d27","d28","d29","d30","d31"])

['MX17004', 2010, '01', 'TMAX', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '-9999', '278', 'S-9999']


ValueError: 35 columns passed, passed data had 37 columns